<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# 必要ライブラリのインストール
! pip install rbot
! pip install json2html
! pip install plotly
! pip install nbformat

In [ ]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

In [ ]:
# rbotのBinanceMarketクラスのインポート
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig, BinanceMarket

In [ ]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
config = BinanceConfig.BTCUSDT
HTML(json2html.convert(config.__str__()))

In [ ]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

market

In [ ]:
# 過去１日分のデータをダウンロード。ダウンロードしたデータは、marketオブジェクトの属性に格納される。
# Trueを指定すると、再ダウンロードを行う。Falseの場合はローカルに保存されている場合は、再ダウンロードを行わない。
market.download(1, False)

In [ ]:
# OHLCVを計算する。
# start_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
# end_time: 終了時刻(UNIX時間[us])　 0の場合はDBにある最後のレコードまで
# window_sec: OHLCVを計算する時間幅(秒)

ohlcv = market.ohlcv(start_time=0, end_time=0, window_sec=60)

In [ ]:
# ohlcvはpolarsのDataFrameオブジェクトとして格納されている。
ohlcv

In [ ]:
# OHLCVの表示
import plotly.graph_objects as go

go.Figure(
    data=[
        go.Candlestick(
            x=ohlcv['time_stamp'],
            open=ohlcv['open'],
            high=ohlcv['high'],
            low=ohlcv['low'],
            close=ohlcv['close']
        )
    ]
).show()


In [ ]:
# VAP(Volume At Price)を計算する。
# from_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
# 戻り値はprice_unit毎に集計されたVAPのPolars DataFrameオブジェクト
vap = market.vap(start_time=0, end_time=0, price_unit=10)

vap

In [ ]:
go.Figure(
    data=[
        go.Scatter(
            x=vap['size'],           
            y=vap['price']
        )
    ]
).show()